In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

# Softmax
## Convert numbers to probabilities with softmax.

$$ P(class=i) = \frac{e^i}{\sum e^i} $$

In [12]:
z = torch.FloatTensor([1, 2, 3]); z

tensor([1., 2., 3.])

In [20]:
hypothesis = torch.softmax(z, dim = 0); hypothesis

tensor([0.0900, 0.2447, 0.6652])

In [16]:
hypothesis.sum()

tensor(1.)

In [1]:
1 + 

0.16666666666666666

In [2]:
2.7 * 2.7

7.290000000000001

In [7]:
2.7/(2.7 + (2.7 * 2.7) + (2.7 * 2.7 * 2.7))

0.09099181073703366

# Cross Entropy Loss (Low-level)
## For multi-class classification, we use the cross entropy loss.

$$ L = \frac{1}{N} \sum - y \log(\hat{y}) $$
where $\hat{y}$ is the predicted probability and $y$ is the correct probability (0 or 1).

In [21]:
z = torch.rand(3, 5, requires_grad = True); z

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)

In [22]:
hypothesis = F.softmax(z, dim = 0); hypothesis

tensor([[0.3412, 0.2938, 0.2671, 0.4002, 0.2469],
        [0.3559, 0.3306, 0.3796, 0.3393, 0.3719],
        [0.3029, 0.3756, 0.3533, 0.2605, 0.3812]], grad_fn=<SoftmaxBackward>)

In [23]:
hypothesis = F.softmax(z, dim = 1); hypothesis 

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)

In [46]:
y = torch.randint(5, (3, )); y

tensor([2, 2, 3])

In [47]:
y.type()

'torch.LongTensor'

In [48]:
y.long(), y.long().type()

(tensor([2, 2, 3]), 'torch.LongTensor')

In [49]:
y_one_hot = torch.zeros_like(hypothesis); y_one_hot

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [63]:
y.unsqueeze(1)

tensor([[2],
        [2],
        [3]])

In [65]:
y.unsqueeze(0)

tensor([[2, 2, 3]])

In [68]:
y.dim()

1

In [66]:
y.shape, y.unsqueeze(1).shape, y.unsqueeze(0).shape

(torch.Size([3]), torch.Size([3, 1]), torch.Size([1, 3]))

In [75]:
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])

In [70]:
y_one_hot.scatter_(1, y.view(3,1), 1)

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])

In [76]:
torch.log(hypothesis)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [77]:
y_one_hot * torch.log(hypothesis)

tensor([[-0.0000, -0.0000, -1.6846, -0.0000, -0.0000],
        [-0.0000, -0.0000, -1.4602, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, -1.8360, -0.0000]], grad_fn=<MulBackward0>)

In [85]:
(y_one_hot * torch.log(hypothesis)).sum(dim = 0)

tensor([ 0.0000,  0.0000, -3.1448, -1.8360,  0.0000], grad_fn=<SumBackward1>)

In [83]:
(y_one_hot * torch.log(hypothesis)).sum(dim = 1)

tensor([-1.6846, -1.4602, -1.8360], grad_fn=<SumBackward1>)

In [81]:
cost = (y_one_hot * torch.log(hypothesis)).sum(dim = 1).mean(); `cost

tensor(-1.6603, grad_fn=<MeanBackward0>)

# Cross-entropy Loss with torch.nn.functional
## PyTorch has F.log_softmax() function.

In [86]:
torch.log(F.softmax(z, dim = 1))

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)

In [89]:
F.log_softmax(z, dim = 1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [91]:
# negative log likely   hood
F.nll_loss(F.log_softmax(z, dim = 1), y)

tensor(1.6603, grad_fn=<NllLossBackward>)

In [98]:
F.cross_entropy(z, y)

tensor(1.6603, grad_fn=<NllLossBackward>)

## Training with Low_level Cross Entropy Loss

In [99]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [105]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W, b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    hypothesis = F.softmax(x_train.matmul(W) + b, dim = 1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim = 1))).sum(dim = 1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


## Training with F.cross_entropy

In [111]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W, b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    hypothesis = x_train.matmul(W) + b
    cost = F.cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## High-level Implementation with nn.Module

In [112]:
class softmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)
        
    def forward(self, x):
        return self.linear(x)

In [113]:
model = softmaxClassifierModel()

In [116]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    
    cost = F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.199255
Epoch  100/1000 Cost: 0.631908
Epoch  200/1000 Cost: 0.551732
Epoch  300/1000 Cost: 0.498875
Epoch  400/1000 Cost: 0.455432
Epoch  500/1000 Cost: 0.416569
Epoch  600/1000 Cost: 0.380047
Epoch  700/1000 Cost: 0.344396
Epoch  800/1000 Cost: 0.308487
Epoch  900/1000 Cost: 0.272014
Epoch 1000/1000 Cost: 0.242334
